In [1]:
# Import libraries
import numpy as np
import os
from PIL import Image
import random

# Set random seed
random.seed(42)

# Unload all data such that it's easily accessible
main_data_path = "/content/drive/MyDrive/ENGSCI/4TH YEAR/fall 4th year/ESC499/Code Test/Finetuning/Data/"
training_data_path = main_data_path+"training"
test_data_path = main_data_path+"test"

training_seg_path = main_data_path+"train_seg_new_bbounds.npz"
test_seg_path = main_data_path+"test_seg_new_bbounds.npz"

training_bb_path = main_data_path+"train_seg_new_beg_end.npz"
test_bb_path = main_data_path+"test_seg_new_beg_end.npz"

In [2]:
# 42s 44s
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


In [3]:
!pip install --upgrade torch torchvision torchaudio # Upgrading PyTorch to the latest version
import torchvision
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from transformers import TextStreamer
from transformers import TrainerCallback, TrainingArguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.1+cu124
    Uninstalling torchaudio-2.5.1+cu124:
      Successfully uninstalled torchaudio-2.5.1+cu124
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Mount the Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# Functions (TEST VERSION)

def load_scan_from_npz(file_path):
    data = np.load(file_path)
    return data['voxel'], data['ax'], data['sag'], data['cor'], data['label']

def pair_segs(filename, patient_specific_bb, patient_bbb):
    v, a, s, c, l = load_scan_from_npz(filename)
    begin, end = patient_bbb
    image_seg_pairs = []
    for i in range(begin, end, 1):
        row_min, col_min, row_max, col_max = patient_specific_bb[i]
        row_mid = int(round((row_min + row_max) / 2))
        col_mid = int(round((col_min + col_max) / 2))
        np_array, label = v[i], l
        np_array = np.uint8(255 * (np_array - np.min(np_array)) / (np.max(np_array) - np.min(np_array)))
        image = Image.fromarray(np_array)
        image_seg_pairs.append({"image":image, "segcen":[(row_mid, col_mid)], "bb":[(row_min, col_min), (row_min, col_max), (row_max, col_max), (row_max, col_min)]})
    return image_seg_pairs

def load_normal_npz(file_path):
    loaded_data = np.load(file_path, allow_pickle = True)
    return loaded_data['array']


def convert_to_conversation(sample):
    instruction = '''
You are an expert medical AI assistant specializing in glioma segmentation on FLAIR-mode brain scans.
Given a 128x128 grayscale brain scan, output the center point of the tumor as a single coordinate (row, col).
The tumor region is the brightest, high-intensity abnormality distinct from normal brain structures.
Ensure the predicted center point accurately represents the geometric center of the tumor, which is generally round in shape.
The output must be formatted strictly as:(row, col) where row and col are integers between 0 and 127, with (0,0) at the top-left, row increasing downward, and col increasing rightward.
Do not output any other text or explanation, only the coordinate in the exact format above.
    '''

    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample['image']} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type": "text", "text": f"{sample['segcen']}"} ]
        },
    ]
    return { "messages" : conversation }


def create_conversation_dataset(data_path, segs_path, bb_path):

    segs_bb = load_normal_npz(segs_path)
    beg_end = load_normal_npz(bb_path)

    # Extract all the patients and the corresponding filenames
    filenames = []
    for filename in os.listdir(data_path):
        if filename.endswith(".npz"):
            file_path = os.path.join(data_path, filename)
            filenames.append(file_path)
    filenames = sorted(filenames, key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Now we build the dataset
    patients = []
    for index in range(len(filenames)):
        filename = filenames[index]
        patient_specific_bb = segs_bb[index]
        patient_bbb = beg_end[index]
        # print(patient_bbb)
        image_seg_pairs = pair_segs(filename, patient_specific_bb, patient_bbb)
        patients.append(image_seg_pairs)
        # patients += image_seg_pairs
    print(patients[0][0])

    # Now convert the dataset into input for LLM
    llm_patients = []
    for patient in patients:
        llm_patient = [convert_to_conversation(sample) for sample in patient]
        llm_patients.append(llm_patient)
    print(f"Number of patients: {len(llm_patients)}")
    print(llm_patients[0][0])

    return patients, llm_patients


In [6]:
data_patients, llm_patients = create_conversation_dataset(test_data_path, test_seg_path, test_bb_path)

{'image': <PIL.Image.Image image mode=L size=128x128 at 0x785A990B0510>, 'segcen': [(40, 100)], 'bb': [(30, 88), (30, 111), (51, 111), (51, 88)]}
Number of patients: 55
{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': '\nYou are an expert medical AI assistant specializing in glioma segmentation on FLAIR-mode brain scans.\nGiven a 128x128 grayscale brain scan, output the center point of the tumor as a single coordinate (row, col).\nThe tumor region is the brightest, high-intensity abnormality distinct from normal brain structures.\nEnsure the predicted center point accurately represents the geometric center of the tumor, which is generally round in shape.\nThe output must be formatted strictly as:(row, col) where row and col are integers between 0 and 127, with (0,0) at the top-left, row increasing downward, and col increasing rightward.\nDo not output any other text or explanation, only the coordinate in the exact format above.\n    '}, {'type': 'image', 'image': <PI

In [7]:
# Add the name of the model we want to import
# lora_model_name = "liufelic/seg_100step_model"
lora_model_name = "liufelic/segcen_200step_model"
# lora_model_name = "unsloth/Llama-3.2-11B-Vision-Instruct"

# Load the model we previously trained
from unsloth import FastVisionModel
model, tokenizer = FastVisionModel.from_pretrained(
    model_name = lora_model_name, # YOUR MODEL YOU USED FOR TRAINING
    load_in_4bit = True, # Set to False for 16bit LoRA
)
FastVisionModel.for_inference(model) # Enable for inference!

==((====))==  Unsloth 2025.2.15: Fast Mllama vision patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/375k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MllamaForConditionalGeneration(
      (vision_model): MllamaVisionModel(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
        (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
          (tile_embedding): Embedding(9, 8197120)
        )
        (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (transformer): MllamaVisionEncoder(
          (layers): ModuleList(
            (0-12): 13 x MllamaVisionEncoderLayer(
              (self_attn): MllamaVisionSdpaAttention(
               

In [8]:
import re

def extract_choice(output_text):
    cleaned_line = output_text.split("assistant")[-1].replace("\n", "||").strip()
    match = re.search(r'\((\d+),\s*(\d+)\)', cleaned_line)
    if match: return (int(match.group(1)), int(match.group(2)))
    else: return (-1, -1)

def write_file_for_one_patient(patient, path):
    # Open the file in append mode (if it doesn't exist, it will be created)
    with open(path, 'w') as file:
      for sample_id in range(len(patient)):

          # Extract the info from the message
          sample = patient[sample_id]
          image = sample["image"]
          ground_truth = sample["segcen"]
          ground_truthbb = sample["bb"]

          FastVisionModel.for_inference(model) # Enable for inference!

          instruction = '''
          You are an expert medical AI assistant specializing in glioma segmentation on FLAIR-mode brain scans.
          Given a 128x128 grayscale brain scan, output the center point of the tumor as a single coordinate (row, col).
          The tumor region is the brightest, high-intensity abnormality distinct from normal brain structures.
          Ensure the predicted center point accurately represents the geometric center of the tumor, which is generally round in shape.
          The output must be formatted strictly as: "(row, col)" where row and col are integers between 0 and 127, with (0,0) at the top-left, row increasing downward, and col increasing rightward.
          Do not output code steps or any other text or explanation, only reply with the coordinate in the exact format above.
          '''

          messages = [
              {"role": "user", "content": [
                  {"type": "image"},
                  {"type": "text", "text": instruction}
              ]}
          ]
          input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
          inputs = tokenizer(
              image,
              input_text,
              add_special_tokens = False,
              return_tensors = "pt",
          ).to("cuda")

          output_tokens = model.generate(**inputs, max_new_tokens=40,
                                          use_cache=True, temperature=1.5, min_p=0.1)
          generated_text = extract_choice(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

          # Prepare the output message
          result_message = f"Ground truth centre: {ground_truth} || Ground truth bbox: {ground_truthbb} || Model output: {generated_text}"

          # # Print to the terminal
          # print(result_message)

          # Write to the file
          file.write(result_message + '\n')  # Add a newline after each entry

    return


In [9]:
# Unload all data such that it's easily accessible
main_results_folder = "/content/drive/MyDrive/ENGSCI/4TH YEAR/fall 4th year/ESC499/Code Test/Finetuning/Results/"

FastVisionModel.for_inference(model)

# for patient_id in range(len(data_patients)):
for patient_id in range(42, 55, 1):

    print(f"Patient ID: {patient_id}")
    patient = data_patients[patient_id]

    # path for patient data storage
    patient_results_text = main_results_folder + f"patient_{patient_id}.txt"
    write_file_for_one_patient(patient, patient_results_text)

Patient ID: 42
Patient ID: 43
Patient ID: 44
Patient ID: 45
Patient ID: 46
Patient ID: 47
Patient ID: 48
Patient ID: 49
Patient ID: 50
Patient ID: 51
Patient ID: 52
Patient ID: 53
Patient ID: 54
